# 如何使用 TVM Pass Infra

**原作者**: [Zhi Chen](https://github.com/zhiics)

随着在 Relay/tir 中优化 pass 数量的增加，手动执行它们并维护它们的依赖关系变得非常棘手。因此，TVM 引入了 infrastructure 来管理优化 pass，并使其适用于 TVM 堆栈中 IR 的不同层。

Relay/tir 程序的优化可以应用在不同的粒度上，即函数级 {py:class}`tvm.relay.transform.FunctionPass`/{py:class}`tvm.tir.transform.PrimFuncPass` 和模块级 {py:class}`tvm.transform.ModulePass`。或者用户可以依赖于 {py:class}`tvm.transform.Sequential` 在 Relay/tir 程序上应用 pass 序列，其中 pass 之间的依赖性可以由 `pass infra` 解析。有关每种 pass 的详细信息，请参阅 {ref}`pass-infra`。

本教程主要演示开发人员如何使用 pass infra 执行某种优化，并为 Relay 程序创建优化管道。同样的方法也可以用于 tir。

In [1]:
import numpy as np
import tvm
from tvm import relay

## 创建 Relay 程序示例

首先，为创建简单的 Relay 程序。本教程中的示例将使用这个程序进行各种优化。类似地，用户可以编写 tir 原语函数并应用 tir passes。

In [2]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    conv = relay.nn.conv2d(x, weight, kernel_size=(3, 3))
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

## 优化程序

现在要优化程序。Relay 具有许多优化功能。选择其中一些应用到这个示例程序中。

有多种方法来优化 Relay 程序。下面将分别提供它们的示例。

### 手动应用优化 passes

创建 Relay 模块，它包含一个或多个用于优化的 Relay 函数。

In [3]:
f = example()
mod = tvm.IRModule.from_expr(f)
mod.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


可以对模块应用常量折叠。

`fold_const` 是不带任何参数的回调函数。

In [4]:
fold_const = relay.transform.FoldConstant()

对给定的模块调用 pass。注意，常量折叠传递在函数级工作。也就是说，模块中的每个函数都将被优化应用。用户不需要手动遍历各个函数来应用此传递。

In [5]:
mod = fold_const(mod)
# 可以从更新的程序中看到，常数是折叠的。
mod.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


以类似的方式应用更多的优化。例如，可以消除  `z` 和 `z1` 使用的常见表达式。

In [6]:
mod = relay.transform.EliminateCommonSubexpr()(mod)
mod.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


一些优化，如融合，也是参数化的。例如，opt 级别 0 将不允许算子融合在一起。用户可以通过 `fuse_opt_level` 来启用它。

In [7]:
mod = relay.transform.FuseOps(fuse_opt_level=0)(mod)

# 可以观察到，优化后的模块包含的函数只有 primitive op
mod.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


### 使用 Sequential 来应用 Passes 序列

像上面那样应用 pass 实际上是很乏味的，它可能需要用户更好地理解它们之间的依赖关系。例如，fusion 目前在 let 绑定上不能很好地工作。因此，如果在融合之前应用  {py:func}`relay.transform.ToANormalForm`，将无法融合可融合的算子，因为此 pass 为每个表达式生成 let 绑定，以规范化 Relay 程序。

因此，Relay 提供了 {py:class}`tvm.transform.Sequential`，通过指定每个 pass 所需的 passes 并将它们打包为一个整体来执行，从而使开发人员不必明确地处理这些问题。例如，现在可以使用 sequential 样式应用相同的 pass，如下所示。{py:class}`tvm.transform.Sequential` 与 [torch.nn.sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) 和 [mxnet.gluon.block](https://mxnet.apache.org/api/python/docs/_modules/mxnet/gluon/block.html) 类似。

例如，`torch.nn.sequential` 用于包含 PyTorch 模块序列，这些模块将被添加以构建网络。它主要关注网络层。相反，pass infra 中的 {py:class}`tvm.transform.Sequential` 作用于优化 pass。

下面通过 {py:class}`tvm.transform.Sequential` 执行一些传递：

In [8]:
f = example()
mod = tvm.IRModule.from_expr(f)
# Glob 感兴趣的 passes.
seq = tvm.transform.Sequential([
    relay.transform.FoldConstant(),
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.FuseOps(fuse_opt_level=2),
])
mod1 = seq(mod)
mod1.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


从变换后的 Relay 程序中，可以看到仍然有两个相同的加法运算。这是因为实际上并没有执行 ``EliminateCommonSubexpr``。原因是在 {py:class}`tvm.transform.Sequential` 下，默认只执行优化级别小于或等于 2 的传递。然而，pass infra 为用户提供了配置接口，以定制他们想要执行的优化级别。

In [9]:
with tvm.transform.PassContext(opt_level=3):
    mod2 = seq(mod)
mod2.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


现在可以看到，两个相同的加法项中只有一个被保留了。

此外，用户可以使用 `disabled_pass` 配置有选择地禁用一些传递，这类似于通用编译器（如 Clang 和 GCC）使用的 `-fno-xxx` 选项。例如，可以如下所示禁用 `EliminateCommonSubexpr`。打印的模块将再次显示两个相同的加法运算。

In [10]:
with tvm.transform.PassContext(opt_level=3, disabled_pass=["EliminateCommonSubexpr"]):
    mod3 = seq(mod)
mod3.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


## 使用 Python 装饰器实现 Pass

下一个示例演示了如何使用 Python 装饰器借助 pass infra 编排定制的优化管道。这个功能大大简化了 pass 的实现。例如，用户可以简单地定义装饰类来进行函数级优化，如下面的示例所示。`transform_function` 包装类，用 `c` 的倍数替换所有常量。稍后，将访问给定模块中的每个函数，并在调用自定义传递时替换函数中的每个常量。

In [11]:
@relay.transform.function_pass(opt_level=1)
class CustomPipeline:
    """简单的测试函数，用于将一个参数替换为另一个参数。"""

    def __init__(self, multiplier):
        self.multiplier = multiplier

    # 这个函数可以定义一个传递。
    def transform_function(self, func, mod, ctx):
        obj = self

        class ReplaceConstant(tvm.relay.ExprMutator):
            def visit_constant(self, c):
                return relay.multiply(obj.multiplier, c)

        return ReplaceConstant().visit(func)


f = example()
mod = tvm.IRModule.from_expr(f)
custom_pass = CustomPipeline(multiplier=relay.const(3, "float32"))
assert custom_pass.info.name == "CustomPipeline"
mod3 = custom_pass(mod)
mod3.show()

/media/pc/data/lxw/ai/tvm/xinetzone/__pypackages__/3.10/lib/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/media/pc/data/tmp/cache/conda/envs/tvmz/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


## 调试 Pass

TVM 为用户提供了即插即用风格（plug-and-play）的调试传递，它通过特殊的传递（ ``PrintIR`` ） 转储（dump）整个模块的 IR，在完成某个传递后打印 IR。对 sequential 传递示例稍加修改的版本如下所示，以便为 ``FoldConstant`` 优化启用 IR 转储。

In [12]:
f = example()
mod = tvm.IRModule.from_expr(f)
seq = tvm.transform.Sequential([
    relay.transform.FoldConstant(),
    tvm.transform.PrintIR(),
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.FuseOps(),
])

通过在 ``FoldConstant`` 之后插入 ``PrintIR``，当 ``FoldConstant`` 完成时，pass infra 将转储模块 IR。用户可以在想要调试的任何 pass 之后插入它，以查看优化效果。


有一个更灵活的调试机制。可以实现 ``PassInstrument`` 类来执行任意代码，不仅在每次传递之前和/或之后，还可以在进入/退出 ``PassContext`` 时执行。查看 {ref}`pass_instrument_cpp_backend` 了解更多信息。

这里使用 {py:func}`tvm.instrument.pass_instrument` 装饰器，实现 PassInsturment 类在每次 Pass 执行前打印 IR：

In [13]:
@tvm.instrument.pass_instrument
class PrintIR:
    """仅在传递执行之前，打印传递名称、IR。"""

    def run_before_pass(self, mod, info):
        print("Running pass: {}", info)
        print(mod)


with tvm.transform.PassContext(opt_level=3, instruments=[PrintIR()]):
    with tvm.target.Target("llvm"):
        # 执行优化。
        mod = seq(mod)
mod.show()

Running pass: {} The meta data of the pass - pass name: sequential, opt_level: 0, required passes: []

def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0], kernel_size=[3, 3]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}


Running pass: {} The meta data of the pass - pass name: FoldConstant, opt_level: 2, required passes: []

def @main(%x: Tensor[(1, 64, 56, 56), float32], %weight: Tensor[(64, 64, 3, 3), float32]) {
  %0 = add(meta[relay.Constant][0], meta[relay.Constant][0]);
  %1 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0], kernel_size=[3, 3]);
  %2 = multiply(%0, 2f);
  %3 = add(%1, %2);
  %4 = add(%3, meta[relay.Constant][0]);
  %5 = add(%3, meta[relay.Constant][0]);
  add(%4, %5)
}


Running pass: {} The meta data of the pass - pas

[20:42:49] /media/pc/data/lxw/ai/tvm/src/relay/ir/function.cc:141: PrintIR():
#[version = "0.0.5"]
def @main(%x: Tensor[(1, 64, 56, 56), float32] /* ty=Tensor[(1, 64, 56, 56), float32] */, %weight: Tensor[(64, 64, 3, 3), float32] /* ty=Tensor[(64, 64, 3, 3), float32] */) -> Tensor[(1, 64, 54, 54), float32] {
  %0 = nn.conv2d(%x, %weight, padding=[0, 0, 0, 0], kernel_size=[3, 3]) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %1 = add(%0, meta[relay.Constant][0] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %2 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  %3 = add(%1, meta[relay.Constant][1] /* ty=Tensor[(1, 64, 54, 54), float32] */) /* ty=Tensor[(1, 64, 54, 54), float32] */;
  add(%2, %3) /* ty=Tensor[(1, 64, 54, 54), float32] */
}

/* For debugging purposes the metadata section has been omitted.
 * If you would like to see the full metadata section you can set the 
 * op

## 小结

本教程介绍了如何使用 pass infra 更方便地在 TVM 中编写和调用 pass。本文还讨论了调用 pass 的不同方法。使用 {py:class}`tvm.transform.Sequential` 可以很大程度上帮助用户简化处理多个优化传递及其依赖关系的工作。此外，还提供了示例来说明如何使用 ``PrintIR`` 和跟踪调试 pass。